In [5]:
from os import path, getcwd
from hundir_la_flota_con_clases import *
import neat
import random
import pickle

In [6]:
def eval_genomes(genomes, config):
    """
    Instanciamos el número de juegos en base a los individuos de la generación(genomas), y lo hacemos jugar,
    hasta que ganan mientras les recompensamos/castigamos en base a sus acciones.
    """
    nets = []
    ge = []
    almirantes = []
    
    # Generamos las redes neuronales a entrenar (nets) y los Almirantes que jugaran
    for genome_id, genome in genomes:
        genome.fitness = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        almirantes.append(Almirante("IA", True))
        ge.append(genome)
        
    for indice, almirante in enumerate(almirantes):
        # Variable de juego
        posicions_conocidas = [(i, j) for i in range(10) for j in range(10)]
        end = True
        
        while (end):
            # Elegir una coordenada para disparar con la red neuronal
            tablero_enemigo_numpy = almirante.tablero_enemigo_para_IA()
            output = nets[almirantes.index(almirante)].activate(tablero_enemigo_numpy)
            fila, columna = almirante.salida_de_red_a_indice(np.argmax(output))

            # Si es una posición valida disparamos y la eliminamos para la siguiente iteración
            if (fila, columna) in posicions_conocidas:
                resultado_del_disparo = almirante.recibir_disparo(fila, columna)
                posicions_conocidas.remove((fila, columna))

            else:
                # Si la red repite posición se le castiga y disparamos de manera aleatoria
                ge[indice].fitness -= 1
                if len(posicions_conocidas) > 0:
                    fila, columna = posicions_conocidas[-1]
                    posicions_conocidas.pop()
                    resultado_del_disparo = almirante.recibir_disparo(fila, columna)
                    
            # A agua, T tocado, H hundido        
            if resultado_del_disparo == "A":
                ge[indice].fitness -= 1
            elif resultado_del_disparo == "T":
                ge[indice].fitness += 10
            elif resultado_del_disparo == "H":
                ge[indice].fitness += 10

            # Actualizar el mapa con (Tocado, Hundido y Agua)
            almirante.actualizar_tras_disparo(resultado_del_disparo, fila, columna)

            if almirante.soy_ganador():
                ge[indice].fitness += 20
                end = False

In [7]:
def run(config_file):
    """
    Ejecuta el algoritmo NEAT para entrenar una red neuronal para que juegue al hundir la flota.
    :param config_file: dirección del archivo de configuración
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Crear la población de en base a la configutación 
    p = neat.Population(config)

    # Mostrar un resumen al final de cada generación
    p.add_reporter(neat.StdOutReporter(False))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    # Generaciones del entrenamiento
    geneaciones = 200
    nombre_del_archivo_entrenado = "red_entrenada"
    # Ejecutar el entrenamiento
    winner = p.run(eval_genomes, geneaciones)
    with open(f"{nombre_del_archivo_entrenado}.pkl", "wb") as f:
        pickle.dump(winner, f)
        
    # Estadisticas finales
    print('\nEl mejor genoma:\n{!s}'.format(winner))

In [ ]:
if __name__ == '__main__':
    # Abrimos el archivo de configuración para el entrenamiento
    config_path = path.join(getcwd(), 'config-feedforward.txt')
    run(config_path)